<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/final_bai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install underthesea

In [2]:
from underthesea import word_tokenize
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import glob
print(glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt"))

['/content/drive/MyDrive/NLP/chatbotdata/giải trí.txt', '/content/drive/MyDrive/NLP/chatbotdata/du lịch.txt', '/content/drive/MyDrive/NLP/chatbotdata/shoping.txt', '/content/drive/MyDrive/NLP/chatbotdata/gia đình.txt', '/content/drive/MyDrive/NLP/chatbotdata/tán gẫu.txt', '/content/drive/MyDrive/NLP/chatbotdata/các câu hỏi phức tạp.txt', '/content/drive/MyDrive/NLP/chatbotdata/đất nước.txt', '/content/drive/MyDrive/NLP/chatbotdata/nghỉ lễ.txt', '/content/drive/MyDrive/NLP/chatbotdata/học tập.txt', '/content/drive/MyDrive/NLP/chatbotdata/thông tin cá nhân.txt', '/content/drive/MyDrive/NLP/chatbotdata/sở thích.txt', '/content/drive/MyDrive/NLP/chatbotdata/trò chuyện về đi ăn.txt', '/content/drive/MyDrive/NLP/chatbotdata/robot.txt', '/content/drive/MyDrive/NLP/chatbotdata/địa chỉ.txt', '/content/drive/MyDrive/NLP/chatbotdata/người yêu.txt', '/content/drive/MyDrive/NLP/chatbotdata/tdtu.txt', '/content/drive/MyDrive/NLP/chatbotdata/bạn bè.txt', '/

In [4]:
all_path = glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt")

In [5]:
FRIEND_PRE_DATA = list()
VOCAB_SIZE = 0
# file = ['bạn bè', 'du lịch', 'gia đình', 'giải trí','học tập', 'nghề nghiệp', 'người yêu','sở thích','tán gẫu','tdtu', 'đất nước', 'địa chỉ','các câu hỏi phức tạp']

In [7]:
for path in all_path:
  with open(path) as frienddata:
    for data in frienddata:
      sent = data.split("__eou__")[:2]
      if sent[0] == ' ' or sent[1] == ' ':
        continue
      if len(sent) == 2:
        anq = list()
        for subsent in sent:
          anq.append(subsent)
        FRIEND_PRE_DATA.append(anq)

In [8]:
df = pd.DataFrame(FRIEND_PRE_DATA, columns=["Question", "Answer"])

In [9]:
df.head(10)

,Question,Answer
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim Marvel nhất
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng"
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ Noo Phước Thịnh
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn"
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động


In [10]:
document_token = [ " ".join([ e.replace(' ','_') for e in word_tokenize(sent.lower())]) for sent in df['Question']]
document_token_list = [word_tokenize(sent.lower()) for sent in df['Question']]  
question_lower = [question.lower() for question in df['Question']]

In [11]:
df["Tokenize_string"] = document_token
df["Tokenize_list"] = document_token_list
df['Question'] = question_lower

In [12]:
df.head(10)

,Question,Answer,Tokenize_string,Tokenize_list
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim Marvel nhất,bạn thích hãng phim nào nhất ?,"[bạn, thích, hãng, phim, nào, nhất, ?]"
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng",thể_loại phim bạn thích là gì ?,"[thể loại, phim, bạn, thích, là, gì, ?]"
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ Noo Phước Thịnh,bạn có thần_tượng ca_sĩ nào không ?,"[bạn, có, thần tượng, ca sĩ, nào, không, ?]"
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc,bạn có muốn thử vận may vào việc chơi đề ?,"[bạn, có, muốn, thử, vận, may, vào, việc, chơi..."
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn",tết này bạn có muốn đi xem phim không ?,"[tết, này, bạn, có, muốn, đi, xem, phim, không..."
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh,cuối tuần này bạn có rãnh hay không ?,"[cuối, tuần, này, bạn, có, rãnh, hay, không, ?]"
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi,bạn có muốn đi xem phim vào cuối tuần này hay ...,"[bạn, có, muốn, đi, xem, phim, vào, cuối, tuần..."
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi,mai bạn có muốn đi đá banh chung với tụi_mình ...,"[mai, bạn, có, muốn, đi, đá, banh, chung, với,..."
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì,bạn có dự_định đi chơi đâu vào cuối tuần không ?,"[bạn, có, dự định, đi, chơi, đâu, vào, cuối, t..."
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động,bạn cho mình hỏi bạn thích xem phim thể_loại gì ?,"[bạn, cho, mình, hỏi, bạn, thích, xem, phim, t..."


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
countVectorizer_sent = CountVectorizer()
countVectorizer_corpus = CountVectorizer()

In [15]:
model_countVectorizer_sent = countVectorizer_sent.fit(df['Tokenize_string'])
model_countVectorizer_corpus = countVectorizer_sent.fit_transform(df['Tokenize_string'])

In [16]:
tfidfVectorizer_sent = TfidfVectorizer()
tfidfVectorizer_corpus = TfidfVectorizer()

In [17]:
model_tfidfVectorizer_sent  = tfidfVectorizer_sent.fit(df['Tokenize_string'])
model_tfidfVectorizer_corpus = tfidfVectorizer_corpus.fit_transform(df['Tokenize_string'])

In [18]:
binaryVectorizer_sent = CountVectorizer(binary=True)
binaryVectorizer_corpus = CountVectorizer(binary=True)

In [19]:
model_binaryVectorizer_sent = binaryVectorizer_sent.fit(df['Tokenize_string'])
model_binaryVectorizer_corpus = binaryVectorizer_corpus.fit_transform(df['Tokenize_string'])

In [20]:
def ir_countVectorizer(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_countVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_countVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]


In [21]:
def ir_tfidfVectorizer(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_tfidfVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_tfidfVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [22]:
def ir_tfidfVectorizer(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_binaryVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_binaryVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [23]:
def ir_binary(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_tfidfVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_tfidfVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [24]:
w2v_model = Word2Vec(df['Tokenize_list'], size = 300, min_count = 2, workers = 3)

In [25]:
embedding_each_sent = list()
for sent_tokenize in df['Tokenize_list']:
  embedding = list()
  for word in sent_tokenize:
    if word not in w2v_model.wv.vocab:
      #300 dimension is the same as size of output embedding word2vec
      embedding.append(np.random.rand(300))  
    else:
      embedding.append(w2v_model.wv.word_vec(word))
  embedding = np.mean(embedding, axis = 0)
  embedding_each_sent.append(embedding)

In [26]:
df['Embedding_gensim'] = embedding_each_sent

In [27]:
def ir_gensim(question):
  question = question.lower()
  tokenize_q = word_tokenize(question)
  q_embedding = list()
  for word in tokenize_q:
    if word not in w2v_model.wv.vocab:
        #300 dimension is the same as size of output embedding word2vec
      q_embedding.append(np.random.rand(300))  
    else:
      q_embedding.append(w2v_model.wv.word_vec(word))
  q_embedding = np.mean(q_embedding, axis = 0)

  r = list()
  for embedding in df['Embedding_gensim']:
    r.append(cosine_similarity(np.array([q_embedding]), np.array([embedding]))[0][0])
    
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]



In [28]:
def ir():
  while(True):
    q = input("Nhập question: ")
    if q == "q":
      break
  
    print("bot with count_vectorize: ", ir_countVectorizer(q))
    print("bot with tfidf_vectorize: ", ir_tfidfVectorizer(q))
    print("bot with binary_vectorize: ", ir_binary(q))
    print("bot with gensim_embedding: ", ir_gensim(q))

In [29]:
ir()

Nhập question: bạn có crush ai không?
bot with count_vectorize:   Mình có thích một người ^^ 
bot with tfidf_vectorize:   có, mình có crush một người. 
bot with binary_vectorize:   Mình có thích một người ^^ 
bot with gensim_embedding:   Mình có thích một người ^^ 
Nhập question: okay bạn
bot with count_vectorize:   Đoán xem 
bot with tfidf_vectorize:   Mình ở quận 9 thành phố Hồ Chí Minh, ngay chỗ đại học sư phạm kỹ thuật á. 
bot with binary_vectorize:   ở đó có chỗ nào để đi chơi không 
bot with gensim_embedding:   không có gì 
Nhập question: bạn là một người rất vui tính
bot with count_vectorize:   mình thấy bạn của mình toàn những đứa điên điên khùng khùng 
bot with tfidf_vectorize:   Thực sự thì mình đúng là người lười vận động 
bot with binary_vectorize:   mình thấy bạn của mình toàn những đứa điên điên khùng khùng 
bot with gensim_embedding:   Quan điểm của mình là không biết 
Nhập question: bạn có hâm mộ ai không?
bot with count_vectorize:   Không có luôn. 
bot with tfidf_vecto